In [8]:
import tensorflow as tf
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [9]:
def scaled_dot_product_attention(q, k, v, mask):
  print("Applying SELF ATTENTION:")
  print("-------------------------------------------------")
  
  matmul_qk = tf.matmul(q, k, transpose_b=True) 
  print("\nmultiplying QUERIE and KEY:")
  print(matmul_qk)

  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  print("\nTaking the dk size:")
  print(dk)

  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
  print("\nStandarding the process:")
  print(scaled_attention_logits)

  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  
  print("\nApplying SOFTMAX in the Standarded vlaues, CALCULATING attention weights:")
  print(attention_weights)
  output = tf.matmul(attention_weights, v)
  print("\nCalculating the outpu, multipling the attention weights and the VALUE MATRIX:")
  print(output)
  print("-------------------------------------------------")
  return output, attention_weights

In [10]:
y = tf.random.uniform((1, 6, 2))  # (batch_size, encoder_sequence, d_model)
y

<tf.Tensor: shape=(1, 6, 2), dtype=float32, numpy=
array([[[0.2251054 , 0.02708256],
        [0.30044246, 0.4288795 ],
        [0.3827877 , 0.9703628 ],
        [0.44823313, 0.7915255 ],
        [0.51326764, 0.28244972],
        [0.36063552, 0.5849689 ]]], dtype=float32)>

In [4]:
scaled_dot_product_attention(y, y, y, mask=None)

Applying SELF ATTENTION:
-------------------------------------------------

multiplying QUERIE and KEY:
tf.Tensor(
[[[0.50541544 0.9106476  0.5522354  0.458029   0.07258042 0.58931434]
  [0.9106476  1.676973   1.0908767  0.74042755 0.1297733  1.1660134 ]
  [0.5522354  1.0908767  0.8573848  0.2756889  0.07665293 0.9199638 ]
  [0.458029   0.74042755 0.2756889  0.613996   0.06812157 0.28976473]
  [0.07258042 0.1297733  0.07665293 0.06812157 0.01045062 0.08174735]
  [0.58931434 1.1660134  0.9199638  0.28976473 0.08174735 0.9871798 ]]], shape=(1, 6, 6), dtype=float32)

Taking the dk size:
tf.Tensor(2.0, shape=(), dtype=float32)

Standarding the process:
tf.Tensor(
[[[0.35738268 0.6439251  0.39048943 0.32387543 0.05132211 0.41670817]
  [0.6439251  1.185799   0.7713663  0.52356136 0.09176359 0.824496  ]
  [0.39048943 0.7713663  0.6062626  0.19494149 0.05420181 0.65051264]
  [0.32387543 0.52356136 0.19494149 0.43416077 0.04816922 0.20489462]
  [0.05132211 0.09176359 0.05420181 0.04816922 0.007

(<tf.Tensor: shape=(1, 6, 2), dtype=float32, numpy=
 array([[[0.6205936 , 0.49994856],
         [0.6732347 , 0.51955765],
         [0.6620698 , 0.4869053 ],
         [0.58101827, 0.511481  ],
         [0.5758982 , 0.4720261 ],
         [0.6685867 , 0.48788267]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 6, 6), dtype=float32, numpy=
 array([[[0.16313252, 0.21726227, 0.1686237 , 0.15775695, 0.12012132,
          0.17310327],
         [0.15348376, 0.26387322, 0.17434502, 0.1360784 , 0.08836124,
          0.18385836],
         [0.15294847, 0.22385038, 0.18978167, 0.12578236, 0.10926908,
          0.19836807],
         [0.17054634, 0.20824036, 0.1499157 , 0.19043148, 0.1294508 ,
          0.15141527],
         [0.16654098, 0.17341419, 0.16702124, 0.16601671, 0.15938282,
          0.167624  ],
         [0.15153037, 0.22782409, 0.19144306, 0.12260558, 0.10583507,
          0.20076185]]], dtype=float32)>)

In [5]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model,use_bias=False)
    self.wk = tf.keras.layers.Dense(d_model,use_bias=False)
    self.wv = tf.keras.layers.Dense(d_model,use_bias=False)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    print("Batch size = :{}".format(batch_size))
    q = self.wq(q)
    k = self.wk(k)  
    v = self.wv(v)
    print("Querie : \n")  
    print(q)
    print("\n Key : \n")  
    print(k)
    print("\n Value : \n")  
    print(v)

    q = self.split_heads(q, batch_size) 
    k = self.split_heads(k, batch_size)  
    v = self.split_heads(v, batch_size)

    print("\n Querie splited: \n")  
    print(q)  
    print("\n Key splited: \n")  
    print(k)
    print("\n Value splited: \n")  
    print(v)  

    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3]) 
    print("\n Transposing the output self attention \n")
    print(scaled_attention)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  
    print("\n Concating the Transposed output self attention \n")
    print(concat_attention)
    output = self.dense(concat_attention)  
    print("\n Applying into a dense layer the Concat Transposed output self attention \n")
    print(output)
    return output, attention_weights

In [6]:
temp_mha = MultiHeadAttention(d_model=32, num_heads=2)

In [7]:
out, attn = temp_mha(y, k=y, q=y, mask=None)
out.shape, attn.shape

Batch size = :Tensor("strided_slice:0", shape=(), dtype=int32)
Querie : 

Tensor("dense_1/MatMul:0", shape=(1, 6, 32), dtype=float32)

 Key : 

Tensor("dense_1_2/MatMul:0", shape=(1, 6, 32), dtype=float32)

 Value : 

Tensor("dense_2_1/MatMul:0", shape=(1, 6, 32), dtype=float32)

 Querie splited: 

Tensor("transpose:0", shape=(1, 2, 6, 16), dtype=float32)

 Key splited: 

Tensor("transpose_1:0", shape=(1, 2, 6, 16), dtype=float32)

 Value splited: 

Tensor("transpose_2:0", shape=(1, 2, 6, 16), dtype=float32)
Applying SELF ATTENTION:
-------------------------------------------------

multiplying QUERIE and KEY:
Tensor("MatMul:0", shape=(1, 2, 6, 6), dtype=float32)

Taking the dk size:
Tensor("Cast:0", shape=(), dtype=float32)

Standarding the process:
Tensor("truediv:0", shape=(1, 2, 6, 6), dtype=float32)

Applying SOFTMAX in the Standarded vlaues, CALCULATING attention weights:
Tensor("Softmax:0", shape=(1, 2, 6, 6), dtype=float32)

Calculating the outpu, multipling the attention weigh

(TensorShape([1, 6, 32]), TensorShape([1, 2, 6, 6]))

In [8]:
class InterpretableMultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, heads, units, dropout_rate):
        super().__init__()
        d_att = units // heads
        self._queries = tf.keras.layers.Dense(heads*d_att, use_bias=False)
        self._keys = tf.keras.layers.Dense(heads*d_att, use_bias=False)
        self._values = tf.keras.layers.Dense(d_att, use_bias=False)

        self._reshape = tf.keras.layers.TimeDistributed(tf.keras.layers.Reshape([heads, d_att]))
        self._softmax = tf.keras.layers.Softmax()
        
        self._attention = tf.keras.layers.Attention(dropout=dropout_rate)
        self._out_weights = tf.keras.layers.Dense(units, use_bias=False)

    def call(self, inputs):
        query, value, key = inputs, inputs, inputs

        query = self._reshape(self._queries(query))
        key = self._reshape(self._keys(key))

        value = tf.repeat(tf.expand_dims(self._values(value), -2), tf.shape(key)[-2], -2)

        outputs, attention = self._attention(
            [query/tf.sqrt(tf.cast(tf.shape(key)[-1], float)), value, key],
            use_causal_mask=True, return_attention_scores=True
        )

        outputs = self._out_weights(tf.reduce_mean(outputs, -2))
        attention = tf.reduce_mean(attention, -2)

        return outputs, attention

In [9]:
class ScaledDotProductAttention(tf.keras.layers.Layer):
  def __init__(self, dropout_rate=0.0, use_scale=True, **kwargs):
    super(ScaledDotProductAttention, self).__init__(**kwargs)
    self.dropout = tf.keras.layers.Dropout(rate=dropout_rate)
    self.softmax = tf.keras.layers.Softmax(axis=-1)
    self.use_scale = use_scale

  def call(self, queries, keys, values, mask=None):
    # Attention calculation
    attn = tf.matmul(queries, keys, transpose_b=True)

    # Scaling (if enabled)
    if self.use_scale:
      dk = tf.cast(tf.shape(keys)[-1], tf.float32)  # Dimension of key vectors
      attn = attn / tf.math.sqrt(dk)

    # Masking (if provided)
    if mask is not None:
      attn = tf.where(mask, attn, tf.constant(-1e9, dtype=attn.dtype))  # Apply mask

    # Apply softmax and dropout
    attn = self.softmax(attn)
    attn = self.dropout(attn)

    # Context layer
    output = tf.matmul(attn, values)

    return output, attn

In [11]:
teste = ScaledDotProductAttention()
teste(y,y,y)

(<tf.Tensor: shape=(1, 6, 2), dtype=float32, numpy=
 array([[[0.43756613, 0.39426136],
         [0.5368911 , 0.42028242],
         [0.5006426 , 0.4020337 ],
         [0.45021075, 0.37592688],
         [0.44898018, 0.4064455 ],
         [0.49814713, 0.38136753]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 6, 6), dtype=float32, numpy=
 array([[[0.16856927, 0.18274643, 0.16895472, 0.14968768, 0.18063694,
          0.14940494],
         [0.12075994, 0.27230155, 0.190591  , 0.11227559, 0.14429735,
          0.15977456],
         [0.13593362, 0.2320518 , 0.1840606 , 0.13057399, 0.15228319,
          0.1650968 ],
         [0.1579818 , 0.1793215 , 0.17128567, 0.16019563, 0.16054478,
          0.17067055],
         [0.16428305, 0.198596  , 0.17213993, 0.13834433, 0.1839302 ,
          0.14270644],
         [0.13309738, 0.21539623, 0.18280438, 0.14405958, 0.13978541,
          0.18485697]]], dtype=float32)>)

In [28]:
class InterpretableMultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, num_heads, d_model, dropout_rate, use_bias=False, **kwargs):
    super(InterpretableMultiHeadAttention, self).__init__(**kwargs)
    self.num_heads = num_heads
    self.d_model = d_model
    self.d_k = self.d_q = self.d_v = d_model // num_heads
    self.dropout = tf.keras.layers.Dropout(rate=dropout_rate)

    self.v_layer = tf.keras.layers.Dense(self.d_v, use_bias=use_bias)
    self.q_layers = [tf.keras.layers.Dense(self.d_q, use_bias=use_bias) for _ in range(num_heads)]
    self.k_layers = [tf.keras.layers.Dense(self.d_k, use_bias=use_bias) for _ in range(num_heads)]
    self.w_h = tf.keras.layers.Dense(self.d_model, use_bias=use_bias)


  def call(self, queries, keys, values, mask=None):
    heads, attns = [], []
    v = self.v_layer(values)

    for i in range(self.num_heads):
      q = self.q_layers[i](queries)
      k = self.k_layers[i](keys)
      
      head, attn = scaled_dot_product_attention(q, k, v, mask)
      head_dropout = self.dropout(head)
      heads.append(head_dropout)
      attns.append(attn)

    
    head = tf.concat(heads, axis=-1) if self.num_heads > 1 else heads[0]
    attn = tf.concat(attns, axis=-1)

    outputs = tf.reduce_mean(head, axis=-1) if self.num_heads > 1 else head
    outputs = self.w_h(outputs)
    outputs = self.dropout(outputs)

    return outputs, attn


In [29]:
teste = InterpretableMultiHeadAttention(num_heads=2, d_model=32, dropout_rate=0.0)

In [30]:
teste(y,y,y)

Applying SELF ATTENTION:
-------------------------------------------------

multiplying QUERIE and KEY:
Tensor("MatMul:0", shape=(1, 6, 6), dtype=float32)

Taking the dk size:
Tensor("Cast:0", shape=(), dtype=float32)

Standarding the process:
Tensor("truediv:0", shape=(1, 6, 6), dtype=float32)

Applying SOFTMAX in the Standarded vlaues, CALCULATING attention weights:
Tensor("Softmax:0", shape=(1, 6, 6), dtype=float32)

Calculating the outpu, multipling the attention weights and the VALUE MATRIX:
Tensor("MatMul_1:0", shape=(1, 6, 16), dtype=float32)
-------------------------------------------------
Applying SELF ATTENTION:
-------------------------------------------------

multiplying QUERIE and KEY:
Tensor("MatMul_2:0", shape=(1, 6, 6), dtype=float32)

Taking the dk size:
Tensor("Cast_1:0", shape=(), dtype=float32)

Standarding the process:
Tensor("truediv_1:0", shape=(1, 6, 6), dtype=float32)

Applying SOFTMAX in the Standarded vlaues, CALCULATING attention weights:
Tensor("Softmax_1

(<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
 array([[ 0.05401925,  0.01050268, -0.10634179, -0.10470515,  0.04372001,
          0.1187161 ,  0.01003221,  0.08324832, -0.00135261, -0.06724634,
          0.01091498,  0.00532866,  0.05282274, -0.03856877,  0.04868086,
         -0.03000465, -0.01532523, -0.10031773, -0.02339172,  0.02514388,
          0.09412222,  0.01039118, -0.03518482,  0.06050512, -0.03160544,
          0.04403522, -0.03556041,  0.0147457 ,  0.01513099, -0.02689542,
         -0.05717932, -0.07184457]], dtype=float32)>,
 <tf.Tensor: shape=(1, 6, 12), dtype=float32, numpy=
 array([[[0.16048244, 0.16856368, 0.16848388, 0.16875336, 0.15818009,
          0.17553659, 0.16950798, 0.16383226, 0.16537184, 0.16757974,
          0.16937682, 0.16433133],
         [0.1579771 , 0.16561386, 0.1682995 , 0.17305674, 0.15274748,
          0.1823053 , 0.16363184, 0.17835698, 0.1696978 , 0.15741716,
          0.16915044, 0.16174582],
         [0.1613272 , 0.16590622, 0.16769479, 0.1

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers


class VariableSelectionNetwork(tf.keras.layers.Layer):
  def __init__(self, hidden_size, dropout_rate, output_size, input_size=None, additional_context=None, **kwargs):
    super(VariableSelectionNetwork, self).__init__(**kwargs)
    self.hidden_size = hidden_size
    self.input_size = input_size
    self.output_size = output_size
    self.dropout_rate = dropout_rate
    self.additional_context = additional_context

    self.flattened_grn = GatedResidualNetwork(
        hidden_size, input_size=input_size, output_size=output_size, dropout_rate=dropout_rate, additional_context=additional_context
    )
    self.per_feature_grn = [GatedResidualNetwork(hidden_size, dropout_rate=dropout_rate) for _ in range(output_size)]

  def call(self, inputs):
    # Non-static inputs
    if self.additional_context is not None:
      embedding, static_context = inputs
      time_steps = tf.shape(embedding)[1]

      flatten = tf.reshape(embedding, (-1, time_steps, self.hidden_size * self.output_size))
      static_context = tf.expand_dims(static_context, axis=1)

      # Nonlinear transformation with gated residual network
      mlp_outputs = self.flattened_grn((flatten, static_context))
      sparse_weights = tf.nn.softmax(mlp_outputs, axis=-1)
      sparse_weights = tf.expand_dims(sparse_weights, axis=2)

      trans_emb_list = []
      for i in range(self.output_size):
        e = self.per_feature_grn[i](embedding[..., i])  # Select i-th feature
        trans_emb_list.append(e)
      transformed_embedding = tf.concat(trans_emb_list, axis=-1)

      combined = sparse_weights * transformed_embedding
      temporal_ctx = tf.reduce_sum(combined, axis=-1)

    # Static inputs
    else:
      embedding = inputs
      flatten = tf.reshape(embedding, [-1, self.input_size * self.hidden_size])  # Assuming batch_size is the first dimension

      # Nonlinear transformation with gated residual network
      mlp_outputs = self.flattened_grn(flatten)
      sparse_weights = tf.nn.softmax(mlp_outputs, axis=-1)
      sparse_weights = tf.expand_dims(sparse_weights, axis=-1)

      trans_emb_list = []
      for i in range(self.output_size):
        e = self.per_feature_grn[i](embedding[:, i:i + 1, :])  # Select i-th feature channel
        trans_emb_list.append(e)
      transformed_embedding = tf.concat(trans_emb_list, axis=1)

      combined = sparse_weights * transformed_embedding
      temporal_ctx = tf.reduce_sum(combined, axis=1)

    return temporal_ctx, sparse_weights